In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler      

from sklearn.linear_model import LogisticRegression

In [2]:
df_historical = pd.read_csv('../../data/credit_card_historical_data.csv')
df_production = pd.read_csv('../../data/credit_card_production_data.csv')

In [3]:
import alibi_detect
from alibi_detect.cd import KSDrift

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
x_hist = df_historical[['type','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']]
le = LabelEncoder()
x_hist['type_encoded'] = le.fit_transform(x_hist['type'])
x_hist = x_hist.drop(['type'],axis=1)
y_hist = df_historical["isFraud"].values
x_train, x_test, y_train, y_test = train_test_split(
    x_hist, y_hist, stratify=y_hist, random_state=42, test_size=0.1, shuffle=True
    )


/var/folders/38/djv715md7dz__89srf2wkcsh0000gn/T/ipykernel_8271/2814568062.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_hist['type_encoded'] = le.fit_transform(x_hist['type'])


In [5]:
x_prod = df_production[['type','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']]

le = LabelEncoder()
x_prod['type_encoded'] = le.fit_transform(x_prod['type'])
x_prod = x_prod.drop(['type'],axis=1)
y_prod = df_historical["isFraud"].values

/var/folders/38/djv715md7dz__89srf2wkcsh0000gn/T/ipykernel_8271/2778788411.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_prod['type_encoded'] = le.fit_transform(x_prod['type'])


# 1)drift detection

In [6]:
cd = KSDrift(x_hist.to_numpy(), p_val=0.05)
preds = cd.predict(x_prod.to_numpy(), drift_type='batch', return_p_val=True, return_distance=True)
preds

{'data': {'is_drift': 0,
  'distance': array([0.00665324, 0.01226359, 0.00306305, 0.00982665, 0.00659862,
         0.00038482], dtype=float32),
  'p_val': array([0.60047305, 0.03739758, 0.99963665, 0.15512381, 0.61098796,
         1.        ], dtype=float32),
  'threshold': 0.008333333333333333},
 'meta': {'name': 'KSDrift',
  'online': False,
  'data_type': None,
  'version': '0.11.2',
  'detector_type': 'drift'}}

# 2) outliers detection

In [7]:
from alibi_detect.od import IForest

In [8]:
od = IForest(
    threshold=0.,
    n_estimators=100
)

od.fit(x_train)
od.infer_threshold(x_hist,threshold_perc=95)

In [10]:
preds = od.predict(x_hist,return_instance_score=True)
preds

{'data': {'instance_score': array([-0.00912618, -0.17338737, -0.07898043, ..., -0.17209841,
         -0.17090177, -0.16529519]),
  'feature_score': None,
  'is_outlier': array([0, 0, 0, ..., 0, 0, 0])},
 'meta': {'name': 'IForest',
  'online': False,
  'data_type': 'tabular',
  'version': '0.11.2',
  'detector_type': 'outlier'}}

In [21]:
outliers_indices = [idx for idx,x in enumerate(preds['data']['is_outlier']) if x==1]
outliers_indices[:3], len(outliers_indices)

([13, 14, 51], 2631)

# 3) Adversial detection